In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/config.json
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/training_args.bin
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/tokenizer.json
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/tokenizer_config.json
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/model.safetensors
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/special_tokens_map.json
/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk/vocab.txt
/kaggle/input/adm-b/Adim B/faiss_meta.jsonl
/kaggle/input/adm-b/Adim B/faiss.index
/kaggle/input/adm-b/Adim B/paragraphs (2).jsonl
/kaggle/input/mahkeme-dataset/mahkeme_dataset/danistay.m/dm_3328.txt
/kaggle/input/mahkeme-dataset/mahkeme_dataset/danistay.m/dm_0003.txt
/kaggle/input/mahkeme-dataset/mahkeme_dataset/danistay.m/dm_8254.txt
/kaggle/input/mahkeme-dataset/mahkeme_dataset/danistay.m/dm_2470.txt
/kaggle/input/mahkeme-dataset/mahkeme_dataset/danistay

# Important NOTE: The plan is as follows, but on this page we performed manual ground-truth (gold) relevance judging,creating labels using my legal expertise.

 Weak supervision: Instead of humans labeling individually, labels are generated automatically via rules/heuristics or LLMs,
 producing “weak/pseudo” labels, and then the model learns from this noisy data.

 Active learning: The model selects which samples to be labeled (most uncertain/most informative),
 and humans label only those selected; the goal is to improve faster with fewer labels.

 Your work with 21 queries × 20 results serves as the seed/gold set for these two approaches:
 if we later apply weak supervision or active learning, we have a reliable reference set to measure
 if the process is moving correctly or not.


# How will it work?
 Weak-supervised rules bootstrap the first model / initial retrieval version and provide a reasonable top-20 candidates for each case text.

Among this top-20, the truly relevant precedent(s) are marked; this costs much less than labeling thousands of decisions individually and aligns directly with the retrieval objective.

 Using these collected labels, re-ranking or pairwise learning is performed, allowing the system to learn better rankings each iteration.

 # One important risk to watch out for:
If you only select from the model’s top-20 results, the model cannot learn about relevant precedents it never shows (selection bias).
Therefore, it’s useful to occasionally mix in random/diverse cases from outside top-20 for monitoring or to collect “hard negatives.”


# Fine-tuning

Your LegalBERT can capture "general semantic similarity," but what you usually want is legal precedent similarity: that is, to consider texts close even if they don't share the same words but have the same fact pattern, applicable norm, or reasoning leading to the result. Because of this difference, most teams perform domain-MLM + task-specific fine-tuning on retrieval; the purpose of fine-tuning is to align the notion of "similarity" with your specific task definition.

Why do you need case-decision pairs?
Because to optimize the model's similarity output, it needs at least some examples signaling "this pair is similar / this pair is not"; otherwise, it only relies on general language knowledge from MLM. In retrieval literature, training re-ranking (cross-encoder) and bi-encoder models typically requires such relevance signals.

How do 50–200 examples generalize?
There are two critical points here:

You are not learning from scratch: The model is already pre-trained (BERT), and you adapted it to legal language with MLM; 50–200 examples do not teach a new language, they just shift the decision boundary (which signals to weight more). This is the key reason why pretrained models can adapt with little data.

The number of training units is effectively much larger: For 200 cases × (1–3 positives + many negatives per case), the model sees thousands of comparisons; especially in pairwise/contrastive training, the "number of examples" grows like this.

What exactly does fine-tuning change?
The unsupervised embedding produces "general proximity."

Fine-tuning teaches the model ranking behavior like "this decision is more relevant for this case, that one less relevant" (learning-to-rank logic) and enables it to use legally distinctive cues (roles of facts, weights of certain patterns, etc.) more consistently.

# Goal
When a case text (1–2 paragraphs) is given, retrieve the top-20 precedent decisions from the pool of 40k decisions; show 1–3 evidence paragraphs per decision.

# Stage 0 — Preparation
Clean/standardize decision texts (single format).

Split each decision into paragraphs and store each as (case_id, paragraph_id, text).

Keep the court name only as metadata (no retrieval filtering for now).

# Stage 1 — Retrieval demo (unlabeled)
Perform semantic search over the entire paragraph pool using the case text.

Aggregate the best matching paragraphs by case_id to produce a decision score.

Show the user the top-20 decisions; for each decision, display the best 1–3 paragraphs as “evidence”; full text can be opened if requested.

# Stage 2 — Feedback (0/1/2)
For each case, assign a 3-level relevance to the returned top-20 decisions:

0 = irrelevant

1 = similar

2 = highly precedent

Store this feedback per query to gradually build a “relevance” dataset (avoiding labeling thousands of decisions upfront).

# Stage 3 — Weak supervision + learning system (long term)
Generate pseudo-labels with heuristic rules to improve initial ranking.

Regularly update the model/re-ranker with your 0/1/2 labels (relevance feedback + active learning).

# Stage 4 — Rationale/fact extraction (once retrieval stabilizes)
After retrieval stabilizes, move to extracting “sentences/facts influencing the decision” (this layer becomes meaningful and reliable only after correct precedents are retrieved).

When moving to code, the first concrete work package includes: paragraph extraction + paragraph embedding generation + paragraph indexing + aggregation for decision score + top-20 UI + 0/1/2 feedback recording. To decide how to split paragraphs (empty lines, numbered paragraphs, PDF line breaks, etc.), just share a sample decision text format.


# Cell A — Scan files, generate paragraph table
Since inputs are read-only on Kaggle, outputs will be written under /kaggle/working.
The cell below walks through .txt files in folders, generates case_id, reads the text,
splits it into paragraphs using robust_paragraphs(), and writes paragraphs.jsonl.


In [3]:
import os, json, re
from pathlib import Path

DATA_ROOT = Path("/kaggle/input/mahkeme-dataset")  # <- değiştirin
OUT_PATH = Path("/kaggle/working/paragraphs.jsonl")

def sentence_split_tr(text: str) -> list[str]:
    text = re.sub(r"\s+", " ", text.strip())
    sents = re.split(r"(?<=[\.\?\!])\s+", text)
    return [s.strip() for s in sents if s and len(s.strip()) > 0]

def make_passages_from_sentences(sents: list[str], sent_per_passage: int = 4) -> list[str]:
    passages, buf = [], []
    for s in sents:
        buf.append(s)
        if len(buf) >= sent_per_passage:
            passages.append(" ".join(buf).strip())
            buf = []
    if buf:
        passages.append(" ".join(buf).strip())
    return passages

def split_by_blank_lines(text: str) -> list[str]:
    parts = re.split(r"\n\s*\n+", text.strip())
    return [p.strip() for p in parts if p and p.strip()]

def robust_paragraphs(raw_text: str,
                     min_blankline_paras: int = 4,
                     max_single_para_chars: int = 1200,
                     sent_per_passage: int = 4) -> list[str]:
    text = raw_text.replace("\r\n", "\n").replace("\r", "\n").replace("\u00a0", " ").strip()
    text = re.sub(r"\n{3,}", "\n\n", text)

    paras = split_by_blank_lines(text)

    if (len(paras) < min_blankline_paras) or (len(paras) == 1 and len(paras[0]) > max_single_para_chars):
        sents = sentence_split_tr(text)
        paras = make_passages_from_sentences(sents, sent_per_passage=sent_per_passage)

    paras = [p for p in paras if len(p) >= 80]
    return paras

def iter_txt_files(root: Path):
    for p in root.rglob("*.txt"):
        yield p

count_files = 0
count_paras = 0

with OUT_PATH.open("w", encoding="utf-8") as f_out:
    for fp in iter_txt_files(DATA_ROOT):
        count_files += 1
        # case_id: dataset içindeki göreli yol (benzersiz)
        case_id = str(fp.relative_to(DATA_ROOT))
        court = fp.parent.name  # klasör adı metadata
        raw_text = fp.read_text(encoding="utf-8", errors="ignore")

        paras = robust_paragraphs(raw_text)

        for i, ptext in enumerate(paras):
            rec = {
                "case_id": case_id,
                "court": court,
                "paragraph_id": i,
                "text": ptext
            }
            f_out.write(json.dumps(rec, ensure_ascii=False) + "\n")
            count_paras += 1

print("files:", count_files)
print("paragraphs:", count_paras)
print("written:", str(OUT_PATH))


files: 46108
paragraphs: 243838
written: /kaggle/working/paragraphs.jsonl


# (1) We will read the JSONL and embed the texts in batches,
# (2) L2 normalize the embeddings and build a FAISS IndexFlatIP,
# (3) write the index and metadata mapping files under /kaggle/working.


In [4]:
!pip -q install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.3 MB/s eta 0:00:00:00:0100:01


In [5]:
# # --- installs (kaggle'da çoğu zaman transformers/torch var; faiss yoksa açın) ---
# # !pip -q install faiss-cpu

# import json
# import numpy as np
# from pathlib import Path
# from tqdm.auto import tqdm

# import torch
# from transformers import AutoTokenizer, AutoModel

# import faiss

# IN_PATH = Path("/kaggle/working/paragraphs.jsonl")
# OUT_INDEX = Path("/kaggle/working/faiss.index")
# OUT_META  = Path("/kaggle/working/faiss_meta.jsonl")

# MODEL_NAME = "/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk"  # örn: "dbmdz/bert-base-turkish-cased" veya sizin LegalBERT repo/path

# device = "cuda" if torch.cuda.is_available() else "cpu"

# def mean_pooling(model_output, attention_mask):
#     # Mean Pooling - attention mask'i dikkate al
#     token_embeddings = model_output[0]  # (bs, seq, hidden)
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
#     sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
#     return sum_embeddings / sum_mask

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModel.from_pretrained(MODEL_NAME).to(device)
# model.eval()

# # 1) JSONL oku (meta'yı RAM'de tutacağız)
# rows = []
# with IN_PATH.open("r", encoding="utf-8") as f:
#     for line in f:
#         rows.append(json.loads(line))

# texts = [r["text"] for r in rows]

# # 2) Batch embedding
# batch_size = 32
# max_length = 256  # paragraf çok uzunsa kırpacağız; sonra gerekirse 384/512 yaparız

# all_embs = []
# with torch.no_grad():
#     for i in tqdm(range(0, len(texts), batch_size)):
#         batch_texts = texts[i:i+batch_size]
#         enc = tokenizer(
#             batch_texts,
#             padding=True,
#             truncation=True,
#             max_length=max_length,
#             return_tensors="pt"
#         ).to(device)

#         out = model(**enc)
#         emb = mean_pooling(out, enc["attention_mask"])
#         emb = torch.nn.functional.normalize(emb, p=2, dim=1)  # cosine için L2 normalize
#         all_embs.append(emb.cpu().numpy().astype(np.float32))

# X = np.vstack(all_embs).astype(np.float32)
# print("Embeddings shape:", X.shape)

# # 3) FAISS IndexFlatIP (cosine = inner product, çünkü normalize ettik)
# dim = X.shape[1]
# index = faiss.IndexFlatIP(dim)
# index.add(X)
# print("FAISS ntotal:", index.ntotal)

# # 4) index + meta kaydet
# faiss.write_index(index, str(OUT_INDEX))

# with OUT_META.open("w", encoding="utf-8") as f:
#     for idx, r in enumerate(rows):
#         meta = {
#             "faiss_id": idx,
#             "case_id": r["case_id"],
#             "court": r.get("court"),
#             "paragraph_id": r["paragraph_id"],
#             "text": r["text"],
#         }
#         f.write(json.dumps(meta, ensure_ascii=False) + "\n")

# print("Saved:", OUT_INDEX, OUT_META)


# Cell C — Query + aggregation + top-20 + evidence paragraphs


In [11]:
import re, json
import numpy as np
from pathlib import Path
from collections import defaultdict

import torch
from transformers import AutoTokenizer, AutoModel
import faiss

# =========================
# Paths (B hücresindeki çıktılar)
# =========================
INDEX_PATH = Path("/kaggle/input/adm-b/Adim B/faiss.index")
META_PATH  = Path("/kaggle/input/adm-b/Adim B/faiss_meta.jsonl")

# =========================
# Model (B ile aynı)
# =========================
MODEL_NAME = "/kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk" 
device = "cuda" if torch.cuda.is_available() else "cpu"

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

# =========================
# Load FAISS + meta
# =========================
index = faiss.read_index(str(INDEX_PATH))

meta = []
with META_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        meta.append(json.loads(line))

def embed_text(text: str, max_length: int = 256) -> np.ndarray:
    with torch.no_grad():
        enc = tokenizer([text], padding=True, truncation=True, max_length=max_length, return_tensors="pt").to(device)
        out = model(**enc)
        emb = mean_pooling(out, enc["attention_mask"])
        emb = torch.nn.functional.normalize(emb, p=2, dim=1)  # cosine için normalize
    return emb.cpu().numpy().astype(np.float32)

# =========================
# Diversity helpers
# =========================
def jaccard(a: set, b: set) -> float:
    if not a and not b:
        return 1.0
    return len(a & b) / max(1, len(a | b))

def tokenize_for_diversity(text: str) -> set:
    t = text.lower()
    t = re.sub(r"[^0-9a-zçğıöşü\s]", " ", t)
    toks = [x for x in t.split() if len(x) >= 4]
    return set(toks)

def pick_diverse_evidence(hits, k=3, max_sim=0.75):
    # hits: list[(score, meta_row)] sorted desc
    selected, selected_tok = [], []
    for sc, m in hits:
        if len(selected) >= k:
            break
        tok = tokenize_for_diversity(m["text"][:800])
        if any(jaccard(tok, st) >= max_sim for st in selected_tok):
            continue
        selected.append((sc, m))
        selected_tok.append(tok)

    # doldurma
    if len(selected) < k:
        for sc, m in hits:
            if len(selected) >= k:
                break
            if (sc, m) not in selected:
                selected.append((sc, m))
    return selected[:k]

# =========================
# Main search
# =========================
def search_cases(query_text: str,
                 top_k_paras: int = 200,
                 top_k_cases: int = 20,
                 evidence_per_case: int = 3,
                 agg_m: int = 3,
                 agg_weights=(1.0, 0.7, 0.5),
                 max_length: int = 256):
    q = embed_text(query_text, max_length=max_length)
    scores, ids = index.search(q, top_k_paras)
    scores = scores[0].tolist()
    ids = ids[0].tolist()

    case_hits = defaultdict(list)  # case_id -> list[(para_score, meta)]
    for sc, fid in zip(scores, ids):
        if fid < 0:
            continue
        m = meta[fid]
        cid = m["case_id"]
        case_hits[cid].append((float(sc), m))

    ranked = []
    for cid, hits in case_hits.items():
        hits_sorted = sorted(hits, key=lambda x: x[0], reverse=True)
        top = hits_sorted[:agg_m]
        w = agg_weights[:len(top)]
        case_score = sum(w[i] * top[i][0] for i in range(len(top)))
        ranked.append((cid, float(case_score), hits_sorted))

    ranked.sort(key=lambda x: x[1], reverse=True)
    ranked = ranked[:top_k_cases]

    results = []
    for cid, csc, hits_sorted in ranked:
        evidences = pick_diverse_evidence(hits_sorted, k=evidence_per_case, max_sim=0.75)
        results.append({
            "case_id": cid,
            "case_score": csc,
            "evidence": [
                {
                    "para_score": float(s),
                    "paragraph_id": int(e["paragraph_id"]),
                    "text": e["text"]
                }
                for (s, e) in evidences
            ]
        })
    return results

# =========================
# RUN DEMO
# =========================
query_text = """ Bir şirket bir bankadan ticari kredi kullanıyor. Başta taksitleri ödüyor ama işler bozulunca birkaç taksit aksıyor. Banka önce arayıp hatırlatıyor, sonra “borç muaccel oldu” diyerek kredinin tamamını talep ediyor ve hesaplara blokaj koydurmaya çalışıyor.

Şirket “bankanın istediği tutar şişirilmiş, faiz/masraf yanlış” diyor; ayrıca bazı ödemelerin hesaba işlenmediğini iddia ediyor. Banka ise “sözleşme böyle, borç açık” diyerek icra takibine gidiyor. Şirket itiraz edince konu mahkemeye taşınıyor; kredi sözleşmesi, hesap ekstresi ve ödeme kayıtları
"""
results = search_cases(
    query_text,
    top_k_paras=200,
    top_k_cases=20,
    evidence_per_case=3,
    agg_m=3,
    agg_weights=(1.0, 0.7, 0.5),
    max_length=256
)

for i, r in enumerate(results, 1):
    print(f"\n#{i} case_id={r['case_id']}  case_score={r['case_score']:.4f}")
    for ev in r["evidence"]:
        preview = ev["text"][:260].replace("\n", " ")
        print(f"  - p{ev['paragraph_id']} score={ev['para_score']:.4f}: {preview}...")


Some weights of BertModel were not initialized from the model checkpoint at /kaggle/input/legalbertturk/pytorch/default/1/LegalBertTurk and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



#1 case_id=mahkeme_dataset/asliye.t.m/atm_5672.txt  case_score=2.0326
  - p3 score=0.9249: borçlarını ödemek üzere şirkete ait banka hesaplarından paralar çektiğini, ödemelerin yaklaşık 3 yıl düzenli olarak yapıldığını düşünen davacının 2005 yılı temmuz ayına kadar herhangi bir sorun ile karşılaşmadığını, 2005 yılında vergi dairesince şirketin banka...
  - p9 score=0.9244: yetkiye dayanarak şirketin SSK,Vergi Dairesi vs. borçlarını ödemek üzere banka hesabından paralar çektiğini, şirketin ilgili ödemelerinin 3 yıldır düzenli olarak yapıldığını düşünen davacı tarafın, 2005 yılı temmuz ayına kadar herhangi bir sorunla karşılaşmaya...
  - p19 score=0.9212: borçlarını ödemek için banka hesaplarından paralar çekmiş, şirketin ilgili ödemelerinin düzenli yapıldığını düşünen davacı tarafın, 2005 yılı temmuz ayına kadar herhangi bir sorunla karşılaşmamış işlerin sağlıklı yürüdüğüne inandırıldığını, Temmuz 2005 yılında...

#2 case_id=mahkeme_dataset/yargitay.h.m/yhm_2064.txt  case_score=2.0207


# Cell D — 0/1/2 feedback recording (Option A)
# This cell uses the query_text and results variables from the most recently run Cell C;
# it asks you for 0/1/2 for each result and appends to /kaggle/working/feedback.jsonl.


**Aynı query–doküman–label eşleşmelerinden “learning to rank / re-rank” eğitimi için eğitim verisi üretmek (model 2’leri üste, 1’leri alta itecek şekilde öğrenir)**

In [7]:
# import shutil
# from pathlib import Path

# SRC = Path("/kaggle/input/feedback-json-last/feedback (3).jsonl")   # sizin upload yolunuz
# DST = Path("/kaggle/working/feedback.jsonl")

# DST.parent.mkdir(parents=True, exist_ok=True)
# shutil.copy(SRC, DST)

# print("Copied:", SRC, "->", DST, "exists:", DST.exists())


Copied: /kaggle/input/feedback-json-last/feedback (3).jsonl -> /kaggle/working/feedback.jsonl exists: True


In [12]:
import json, time
from pathlib import Path

FEEDBACK_PATH = Path("/kaggle/working/feedback.jsonl")

def show_query_block(query_text, preview_chars=1200):
    print("\n" + "#"*100)
    print("QUERY / OLAY METNİ")
    qt = query_text.strip().replace("\n", " ")
    if len(qt) > preview_chars:
        qt = qt[:preview_chars] + "..."
    print(qt)
    print("#"*100 + "\n")

def show_case_block(i, r, preview_chars=700):
    print("\n" + "="*100)
    print(f"[{i}] case_id={r['case_id']}  case_score={r['case_score']:.4f}")
    for j, ev in enumerate(r["evidence"], 1):
        txt = ev["text"].replace("\n", " ").strip()
        if len(txt) > preview_chars:
            txt = txt[:preview_chars] + "..."
        print(f"  Evidence {j}: (p{ev['paragraph_id']}, score={ev['para_score']:.4f})")
        print(f"  {txt}\n")

def ask_label():
    while True:
        x = input("Label (0=alakasız, 1=benzer, 2=çok emsal) => ").strip()
        if x in {"0","1","2"}:
            return int(x)
        print("Lütfen sadece 0, 1 veya 2 girin.")

query_id = str(int(time.time()))

# 1) Query'i en başta göster
show_query_block(query_text, preview_chars=1400)

# 2) Sonuçları tek tek göster ve label al
records = []
for i, r in enumerate(results, 1):
    show_case_block(i, r, preview_chars=800)
    label = ask_label()
    rec = {
        "query_id": query_id,
        "query_text": query_text,
        "case_id": r["case_id"],
        "label": label,
        "case_score": r["case_score"],
        "evidence": [
            {
                "paragraph_id": e["paragraph_id"],
                "para_score": e["para_score"],
                "text_preview": e["text"][:220].replace("\n", " ")
            }
            for e in r["evidence"]
        ]
    }
    records.append(rec)

with FEEDBACK_PATH.open("a", encoding="utf-8") as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("\nSaved labels:", len(records))
print("query_id:", query_id)
print("file:", str(FEEDBACK_PATH))



####################################################################################################
QUERY / OLAY METNİ
Bir şirket bir bankadan ticari kredi kullanıyor. Başta taksitleri ödüyor ama işler bozulunca birkaç taksit aksıyor. Banka önce arayıp hatırlatıyor, sonra “borç muaccel oldu” diyerek kredinin tamamını talep ediyor ve hesaplara blokaj koydurmaya çalışıyor.  Şirket “bankanın istediği tutar şişirilmiş, faiz/masraf yanlış” diyor; ayrıca bazı ödemelerin hesaba işlenmediğini iddia ediyor. Banka ise “sözleşme böyle, borç açık” diyerek icra takibine gidiyor. Şirket itiraz edince konu mahkemeye taşınıyor; kredi sözleşmesi, hesap ekstresi ve ödeme kayıtları
####################################################################################################


[1] case_id=mahkeme_dataset/asliye.t.m/atm_5672.txt  case_score=2.0326
  Evidence 1: (p3, score=0.9249)
  borçlarını ödemek üzere şirkete ait banka hesaplarından paralar çektiğini, ödemelerin yaklaşık 3 yıl düzenli olarak y

Label (0=alakasız, 1=benzer, 2=çok emsal) =>  1



[2] case_id=mahkeme_dataset/yargitay.h.m/yhm_2064.txt  case_score=2.0207
  Evidence 1: (p3, score=0.9212)
  II. CEVAP Davalı vekili cevap dilekçesinde; müvekkilinin kefaletinin geçersiz olduğunu, kefilin imzası, sorumlu olacağı limit, kefalet tarihi, müteselsil kefil olması durumunda bu sıfatla veya bu anlama gelen herhangi bir ifadeyle yükümlülük altına girdiğinin kefalet sözleşmesinde kendi el yazısıyla belirtmesinin şart olduğunu, davacı bankaca matbu bir metin kullanıldığını, borcun ipotekle temin edilmesi nedeniyle önce ipotekten alacağın karşılanması gerektiğini, davacının borca işletilen faiz oranlarında tek taraflı ve keyfi düzenlemeyle değişiklikler yaptığını, bankanın asıl borçluya gönderdiği ihtarlarla faiz oranı, komisyon oranı vs. değişiklerin kanuna, sözleşmeye uygun olmadığını, alacağın likit sayılamayacağını, icra inkâr tazminatı koşullarının bulunmadığını savunarak davanın reddi...

  Evidence 2: (p4, score=0.9166)
  III. İLK DERECE MAHKEMESİ KARARI İlk Derece Mahkeme

Label (0=alakasız, 1=benzer, 2=çok emsal) =>  1



[3] case_id=mahkeme_dataset/yargitay.h.m/yhm_2967.txt  case_score=2.0135
  Evidence 1: (p3, score=0.9169)
  sayılı kararıyla; kredi sözleşmesinin yapıldığı tarihte kredi kullananın altmış beş yaşın üzerinde olduğu ve okuma yazma bilmediği, sözleşmeye parmak bastığı, bu hâldeyken davalı Banka yetkililerince herhangi bir sağlık raporu alınmadan işlem yapılarak kredi verildiği, alınan rapor ve hastane cevaplarından da ilgilinin uzun süreden beri rahatsız olduğu ve kredi kullandırıldığı sırada akli dengesinin yerinde olmadığı gerekçesiyle, davacının davalı Bankaya 24.05.2010 tarihli kredi sözleşmesi nedeniyle borçlu olmadığının tespitine karar verilmiş; kararı süresi içinde davalı vekili temyiz etmiştir. IV. BOZMA VE BOZMADAN SONRAKİ YARGILAMA SÜRECİ 1. Yargıtay (Kapatılan) 13.

  Evidence 2: (p1, score=0.9151)
  Mahkemece verilen direnme kararı, davalı vekili tarafından temyiz edilmiş ve Yargıtay Hukuk Genel Kurulu tarafından direnme uygun bulunarak, davalı vekilinin sair temyiz itirazla

...


[19] case_id=mahkeme_dataset/yargitay.h.m/yhm_2447.txt  case_score=0.9215
  Evidence 1: (p2, score=0.9215)
  I. DAVA Davacı dava dilekçesinde; ... Finans Kurumu A.Ş.'nin faaliyet izninin Bankalar Kurulu kararına istinaden kaldırıldığını ve şirketin tasfiyeye girdiğini, finans kurumuna 2001 yılından önce dolar ve euro yatırdığını, genel kurul toplantısında küçük meblağdan büyük meblağa doğru sırasıyla ödeme yapılmasının kararlaştırıldığını, ödeme sırasının dışına çıkılacak olunur ise tasfiye memurlarının ve münfesih şirketin hukuki ve cezai sorumlulukları olacağının yetkililer tarafından yazılı olarak beyan edildiğini, 2002 yılında 500,00 TL ödeme yapıldığını, daha sonra 19 yıl boyunca başka bir ödeme yapılmadığını, yüzlerce sıra dışı ödemeler yapıldığını, alacağını tahsil edemediğinden zarara uğradığını ileri sürerek fazlaya ilişkin hakkını saklı kalmak kaydıyla Tasfiye Kurulu Başkanının genel ku...



Label (0=alakasız, 1=benzer, 2=çok emsal) =>  0



[20] case_id=mahkeme_dataset/asliye.t.m/atm_4236.txt  case_score=0.9213
  Evidence 1: (p0, score=0.9213)
  Mahkememizde görülmekte olan Bankalarca Kullandırılan Ticari Kredilerden Ve Ticari Kredili Mevduatlardan Kaynaklanan Davalar (Tazminat) davasının yapılan açık yargılaması sonunda, GEREĞİ DÜŞÜNÜLDÜ: Davacı vekili dava dilekçesinde özetle; davalı Banka’dan 03.09.2024 tarihinde 2.500.000,00 TL tutarında kredi, 04.09.2024 tarihinde ise bilgisi ve rızası dışında 500.000,00 TL tutarında taksitli nakit avans kullandırıldığını, her iki krediyi de kısa süre içerisinde kapatmak istediğinde davalı tarafından erken ödeme ücreti, faiz, KKDF ve BSMV adı altında toplam 326.880,00 TL kesinti yapıldığını, bu tutarların kullandığı kredi miktarının yaklaşık %10’una tekabül ettiğini ve hakkaniyet ile dürüstlük kuralına açıkça aykırı olduğunu ileri sürmüştür. Davacı, sözleşmelerin genel işlem koşulu niteliğind...



Label (0=alakasız, 1=benzer, 2=çok emsal) =>  2



Saved labels: 20
query_id: 1766799906
file: /kaggle/working/feedback.jsonl


# 20 queries × 20 candidate documents per query = 400 judgments with 0/1/2 graded relevance labels obtained
